In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import analysis.yinf as yinf
import analysis.products as products
import analysis.util as util
from data.util import *
import os, sys, re
from scipy.stats import gaussian_kde
ML_DIR = os.path.expandvars("$SCRATCH/mlreco_cell/")
NETS = dict(enumerate(sorted([d for d in os.listdir(os.path.expandvars(ML_DIR))])))
SIM_DIR = os.path.expandvars("$SCRATCH/larsim/")
SIMS = dict(enumerate(sorted([d for d in os.listdir(os.path.expandvars(SIM_DIR)) if "reco" in d])))

def parse_stats_dir(stats_dir = './stats'):
    stats_files = []
    for root, dirs, files in os.walk(stats_dir):
        rel_path = os.path.relpath(root, stats_dir)
        if rel_path.count('/') != 1: continue
        slash_i = rel_path.find('/')
        header = [rel_path[:slash_i], rel_path[slash_i+1:]]
        for f in files:
            epoch, _ = inf_file_info(f)
            thres = re.findall('thres([0-9]*[.]?[0-9]+)', f)
            if len(thres) == 1: 
                thres = float(thres[0])
            else:
                thres = -1
            stats_files.append(header+[epoch, thres, root+'/'+f])
    return pd.DataFrame(stats_files, columns=['net', 'inf', 'epoch', 'thres', 'file'])

STATS = parse_stats_dir().sort_values(by=['epoch'])

def select_stats(net=None, inf=None, thres=None, first=0, last=-1, stride=1, stats_df=STATS):
    if net is not None:
        stats_df = stats_df[stats_df.net==net]
    if inf is not None:
        stats_df = stats_df[stats_df.inf==inf]
    if thres is not None:
        stats_df = stats_df[stats_df.thres==thres]
    if last == -1:
        epochs = sorted(set(stats_df.epoch))[first::stride]
    else:
        epochs = sorted(set(stats_df.epoch))[first:last+1:stride]
    return list(stats_df[stats_df.epoch.isin(epochs)]['file'])


In [ ]:
NETS

In [ ]:
SIMS

In [ ]:
xy_dir = SIM_DIR+"/reco_1GeV_BeamCosmicHit_xy/"
BCHit = {}
for file in sorted(os.listdir(xy_dir))[:100]:
    for event in range(5):
        voxel_truth, voxel_active, _ = products.parse_xy(event, 1, xy_dir+file)
        voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
        BCHit[(file, event)] = util.SP_stats(voxel_truth, voxel_active, False, False)

In [ ]:
Pur_Hit = [x[0] for x in BCHit.values()]
Eff_Hit = [x[1] for x in BCHit.values()]
print(np.mean(Pur_Hit))
print(np.mean(Eff_Hit))

In [ ]:
xy_dir = SIM_DIR+"/reco_1GeV_BeamCosmic_xy/"
BCWire = {}
for file in sorted(os.listdir(xy_dir))[:100]:
    for event in range(5):
        voxel_truth, voxel_active, _ = products.parse_xy(event, 1, xy_dir+file)
        voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
        BCWire[(file, event)] = util.SP_stats(voxel_truth, voxel_active, False, False)

In [ ]:
BCKeys = BCHit.keys() & BCWire.keys()
Pur_Wire = [BCWire[k][0] for k in BCKeys]
Eff_Wire = [BCWire[k][1] for k in BCKeys]
Pur_Hit = [BCHit[k][0] for k in BCKeys]
Eff_Hit = [BCHit[k][1] for k in BCKeys]
print(np.mean(Pur_Wire))
print(np.mean(Eff_Wire))
print(np.mean(Pur_Hit))
print(np.mean(Eff_Hit))
print(np.corrcoef(Eff_Hit, Eff_Wire))
print(np.corrcoef(Pur_Hit, Pur_Wire))

In [ ]:
fig, axes = plt.subplots(2)
axes[0].scatter(Eff_Wire, Eff_Hit, s=2)
axes[1].scatter(Pur_Wire, Pur_Hit, s=2)
for ax in axes:
    ax.set_xlim((0,1))
    ax.set_ylim((0,1))
    ax.set_xlabel("Wire")
    ax.set_ylabel("Hit")
axes[0].set_title("Sensitivity")
axes[1].set_title("Purity")
fig.text(0.2, 0.8, "Wire mean: %.3f"%np.mean(Eff_Wire))
fig.text(0.2, 0.7, "Hit mean: %.3f"%np.mean(Eff_Hit))
fig.text(0.2, 0.3, "Wire mean: %.3f"%np.mean(Pur_Wire))
fig.text(0.2, 0.2, "Hit mean: %.3f"%np.mean(Pur_Hit))
fig.suptitle("BeamCosmic Wire vs. Hit (%d Events)"%len(Eff_Wire))

In [ ]:
xy_dir = SIM_DIR+"/reco_1GeV_ElectronWire_xy-v3_3/"
EWire = {}
for file in [f for f in os.listdir(xy_dir) if file_info(f)[1]>950 and file_info(f)[1]<=960 and batch_info(f)[0]==0]:
    for event in range(50):
        voxel_truth, voxel_active, _ = products.parse_xy(event, 1/3, xy_dir+file)
        voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
        EWire[(file, event)] = util.SP_stats(voxel_truth, voxel_active, False, False)

In [ ]:
Pur = [x[0] for x in EWire.values()]
Sen = [x[1] for x in EWire.values()]
print(np.mean(Pur))
print(np.mean(Sen))

In [ ]:
xy_dir = SIM_DIR+"/reco_1GeV_ElectronHit_xy-v3/"
EHit = {}
for file in [f for f in os.listdir(xy_dir) if file_info(f)[1]>950 and file_info(f)[1]<=960]:
    for event in range(100):
        voxel_truth, voxel_active, _ = products.parse_xy(event, 1/3, xy_dir+file)
        voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
        EHit[(file, event)] = util.SP_stats(voxel_truth, voxel_active, False, False)

In [ ]:
Pur = [x[0] for x in EHit.values()]
Sen = [x[1] for x in EHit.values()]
print(np.mean(Pur))
print(np.mean(Sen))

In [ ]:
EKeys = EHit.keys() & EWire.keys()
Pur_Wire = [EWire[k][0] for k in EKeys]
Eff_Wire = [EWire[k][1] for k in EKeys]
Pur_Hit = [EHit[k][0] for k in EKeys]
Eff_Hit = [EHit[k][1] for k in EKeys]
print(np.mean(Pur_Wire))
print(np.mean(Eff_Wire))
print(np.mean(Pur_Hit))
print(np.mean(Eff_Hit))
print(np.corrcoef(Eff_Hit, Eff_Wire))
print(np.corrcoef(Pur_Hit, Pur_Wire))

In [ ]:
fig, axes = plt.subplots(2)
axes[0].scatter(Eff_Wire, Eff_Hit, s=2)
axes[1].scatter(Pur_Wire, Pur_Hit, s=2)
for ax in axes:
    ax.set_xlim((0,1))
    ax.set_ylim((0,1))
    ax.set_xlabel("Wire")
    ax.set_ylabel("Hit")
axes[0].set_title("Sensitivity")
axes[1].set_title("Purity")
fig.text(0.2, 0.8, "Wire mean: %.3f"%np.mean(Eff_Wire))
fig.text(0.2, 0.7, "Hit mean: %.3f"%np.mean(Eff_Hit))
fig.text(0.2, 0.3, "Wire mean: %.3f"%np.mean(Pur_Wire))
fig.text(0.2, 0.2, "Hit mean: %.3f"%np.mean(Pur_Hit))
fig.suptitle("Electron Wire vs. Hit") 

In [ ]:
index = 11
event = 3
#products.plot_xy(plt.figure(), event, SIM_DIR+"reco_1GeV_BeamCosmicHit_xy/batch0-reco_BeamCosmic_0%d_xy-TPC1.npz"%index)
products.plot_xy(plt.figure(), event, 1, .5/3, 10, SIM_DIR+"reco_1GeV_BeamCosmic_xy/batch0-reco_BeamCosmic_%03d_xy-TPC1.npz"%index)

In [ ]:
products.plot_mc_stats(plt.figure(), "mc/reco_1GeV_BeamCosmicHit_xy-tpc1-mc.npz")

In [ ]:
index = 953
event = 1
products.plot_xy(plt.figure(), event, 1/3, .5/3, 50, xy_file=SIM_DIR+"reco_1GeV_ElectronWire_xy-v3_3/batch0-reco_singleElectron_%03d_xy-TPC1.npz"%index)

In [ ]:
stats_file = "mc/reco_1GeV_BeamCosmic_xy-tpc1-mc.npz"
Sizes_true, Sizes_cluster, Indexes, Tracks, Purs, Effs, Thetas, Phis = [], [], [] ,[], [], [], [], []
PDG_id = [13, -13]
i = 0
with np.load(stats_file, allow_pickle=True) as stats_f:
    for index, event_mc in stats_f.items():
        for mc, coords in event_mc.item().items():
            if not mc[1] in PDG_id: continue
            track_stats = mc[3]
            if track_stats is None: continue
            cluster_size = len(coords[0])
            if cluster_size < 50: continue
                
            Thetas.append(track_stats[0])
            Phis.append(track_stats[1])
            Indexes.append(index)
            Sizes_true.append(mc[2])
            Sizes_cluster.append(cluster_size)
            Tracks.append(mc[0])
            Effs.append((mc[2]-len(coords[1]))/mc[2])
            Purs.append((mc[2]-len(coords[1]))/cluster_size)
fig, axes = plt.subplots(2)
axes[0].scatter(Sizes_true, Effs, s=2)
axes[1].scatter(Sizes_cluster, Effs, s=2)
fig, axes = plt.subplots(2)
axes[0].scatter(Sizes_true, Purs, s=2)
axes[1].scatter(Sizes_cluster, Purs, s=2)

In [ ]:
Sizes_true = np.array(Sizes_true)
Sizes_cluster = np.array(Sizes_cluster)
Indexes = np.array(Indexes)
Tracks = np.array(Tracks)
Purs = np.array(Purs)
Effs = np.array(Effs)
print(np.corrcoef(Sizes_true, Purs))
print(np.mean(Purs), np.mean(Effs))
print(len(Purs))
args = np.argsort(Effs)[:20]
#args = Purs>0.2
print(Sizes_true[args])
print(Sizes_cluster[args])
print(Indexes[args])
print(Tracks[args])
print(Purs[args])
print(Effs[args])

In [ ]:
from scipy import stats
fig, axes = plt.subplots(2)
bin_means, bin_edges, binnumber = stats.binned_statistic(Thetas, Effs, bins=150)
axes[0].scatter(Thetas, Effs, s=2)
axes[0].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
bin_means, bin_edges, binnumber = stats.binned_statistic(Phis, Effs, bins=150)
axes[1].scatter(Phis, Effs, s=2)
axes[1].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
axes[0].set_title("Efficiency vs. Theta (Muon)")
axes[1].set_title("Efficiency vs. Phi (Muon)")

In [ ]:
from scipy import stats
fig, axes = plt.subplots(2)
bin_means, bin_edges, binnumber = stats.binned_statistic(Thetas, Purs, bins=150)
axes[0].scatter(Thetas, Purs, s=2)
axes[0].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
bin_means, bin_edges, binnumber = stats.binned_statistic(Phis, Purs, bins=150)
axes[1].scatter(Phis, Purs, s=2)
axes[1].hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='r', lw=5,
           label='binned statistic of data')
axes[0].set_title("Purity vs. Theta (Muon)")
axes[1].set_title("Purity vs. Phi (Muon)")

In [ ]:
E_missing, E_T_active = [], []
for index in range(701, 710):
    for event in range(50):
        xy_file = SIM_DIR+"reco_1GeV_ElectronWire_xy-v3_3/batch0-reco_singleElectron_%03d_xy-TPC1.npz"%index
        voxel_truth, voxel_active, event_info = products.parse_xy(event, 1/3, xy_file)
        voxel_missing, voxel_T_active, _, _ = util.comp_voxels(voxel_truth, voxel_active)
        E_missing.extend(voxel_missing.values())
        E_T_active.extend(voxel_T_active.values())

In [ ]:
fig, ax = plt.subplots()
Ns, bins, _ = ax.hist([E_missing, E_T_active], histtype='step', color=["r", "b"], label=["True but not Active: N=%d, Sum=%.1f"%(len(E_missing), sum(E_missing)), "True and Active: N=%d, Sum=%.1f"%(len(E_T_active), sum(E_T_active))], bins=np.linspace(0, 3, 16))
ax.text(0.3, 5*10**4, "For E < 0.2MeV, %.1f%% of Voxels Missing from Tiling"%(100*Ns[0][0]/(Ns[0][0]+Ns[1][0])))

ax.set_xticks(bins)
ax.set_yscale('log', basey=10)
ax.set_title("Energy Histo True vs Active")
ax.set_xlabel("Energy [MeV]")
ax.set_ylabel("# Voxels")
ax.legend()